In [12]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## IO markup

### Train

In [2]:
from modules.data import bert_data

In [3]:
train_df_path = "/home/eartemov/ae/work/factRuEval-2016/dev.csv"
valid_df_path = "/home/eartemov/ae/work/factRuEval-2016/test.csv"

In [4]:
data = bert_data.LearnData.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels1.txt",
    clear_cache=True
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [5]:
from modules.models.bert_models import BERTCRF

In [6]:
model = BERTCRF.create(len(data.train_ds.idx2label), crf_dropout=0.3)

In [7]:
from modules.train.train import NerLearner

In [8]:
num_epochs = 100

In [9]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/fre-BERTCRF-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [10]:
model.get_n_trainable_params()

298489

In [ ]:
learner.fit(epochs=num_epochs)

### Predict

In [13]:
from modules.data.bert_data import get_data_loader_for_predict

In [14]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [15]:
preds = learner.predict(dl)

In [16]:
from sklearn_crfsuite.metrics import flat_classification_report

In [17]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [18]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [19]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=data.train_ds.idx2label[5:], digits=4)

In [20]:
print(tokens_report)

              precision    recall  f1-score   support

       I_LOC     0.8576    0.7932    0.8242      1557
       I_PER     0.9544    0.9616    0.9580      2112
       I_ORG     0.8150    0.7490    0.7806      3865

   micro avg     0.8653    0.8178    0.8409      7534
   macro avg     0.8757    0.8346    0.8543      7534
weighted avg     0.8629    0.8178    0.8394      7534



In [21]:
from modules.analyze_utils.main_metrics import precision_recall_f1

In [22]:
results = precision_recall_f1(true_labels, pred_labels)

processed 56409 tokens with 7534 phrases; found: 7120 phrases; correct: 6161.

precision:  86.53%; recall:  81.78%; FB1:  84.09

	LOC: precision:  85.76%; recall:  79.32%; F1:  82.42 1440

	ORG: precision:  81.50%; recall:  74.90%; F1:  78.06 3552

	PER: precision:  95.44%; recall:  96.16%; F1:  95.80 2128


